In [2]:
import pandas as pd
import numpy as np
import os

## Loading and dealing with the metadata table

In [2]:
meta_data = pd.read_csv("41587_2020_603_MOESM3_ESM.csv")

In [17]:
meta_data.head()

,Species representative,MGnify accession,ENA accession,Status,Reference genes,Core genes1,% Core genes1,Accessory genes1,Pan-genome size1,Number of SNVs,...,Unnamed: 16,Unnamed: 17,Number of 97% ANI clusters,Number of 99% ANI clusters,Number of genomes (total),Number of genomes (non-redundant),Number of genomes (near-complete),Number of samples,Taxonomy lineage (GTDB),Taxonomy lineage (NCBI)
0,GUT_GENOME000001,MGYG-HGUT-00001,ERS3638031,Cultured (human gut),3182,NaN,NaN,NaN,NaN,16351.0,...,0,1,2,4,4,3,1,3,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__P...,d__Bacteria;p__Firmicutes;c__Clostridia;o__Clo...
1,GUT_GENOME000004,MGYG-HGUT-00002,ERS3638032,Cultured (human gut),3920,1821.0,46.45,34621.0,36442.0,428560.0,...,49,162,134,327,358,308,120,308,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__L...,d__Bacteria;p__Firmicutes;c__Clostridia;o__Clo...
2,GUT_GENOME000008,MGYG-HGUT-00003,ERS3638033,Cultured (human gut),2690,1522.0,56.58,19714.0,21236.0,516967.0,...,244,676,42,561,1178,1000,593,1003,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,d__Bacteria;p__Bacteroidetes;c__Bacteroidia;o_...
3,GUT_GENOME000010,MGYG-HGUT-00004,ERS3638034,Cultured (human gut),3634,NaN,NaN,NaN,NaN,58094.0,...,5,5,3,17,24,22,8,23,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__O...,d__Bacteria;p__Firmicutes;c__Clostridia;o__Clo...
4,GUT_GENOME000017,MGYG-HGUT-00005,ERS3638035,Cultured (human gut),3755,NaN,NaN,NaN,NaN,NaN,...,0,0,1,2,2,2,1,2,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__P...,d__Bacteria;p__Firmicutes;c__Clostridia;o__Clo...


In [18]:
meta_data.sort_values('Number of genomes (near-complete)', ascending=False, inplace=True)

In [19]:
top_species = meta_data.iloc[:100, -1].apply(lambda x: ' '.join(x.split(';')[-2:]))

In [20]:
def find_species_row(meta_data, species_str):
    mask = meta_data.iloc[:, -1].apply(lambda x: species_str in x)
    return meta_data[mask]

In [21]:
# finding B. vulgatus (B. dorei)
find_species_row(meta_data, 'dorei')

,Species representative,MGnify accession,ENA accession,Status,Reference genes,Core genes1,% Core genes1,Accessory genes1,Pan-genome size1,Number of SNVs,...,Unnamed: 16,Unnamed: 17,Number of 97% ANI clusters,Number of 99% ANI clusters,Number of genomes (total),Number of genomes (non-redundant),Number of genomes (near-complete),Number of samples,Taxonomy lineage (GTDB),Taxonomy lineage (NCBI)
2477,GUT_GENOME143505,MGYG-HGUT-02478,ERS3640508,Cultured (human gut),4517,2259.0,50.01,192070.0,194329.0,842418.0,...,992,2343,276,2853,5750,3649,2090,3685,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,d__Bacteria;p__Bacteroidetes;c__Bacteroidia;o_...


In [22]:
mgnify_accession = find_species_row(meta_data, 'rectale')['MGnify accession'].squeeze()
print("E. rectale's accession is {}".format(mgnify_accession))

E. rectale's accession is MGYG-HGUT-02492


## Downloading ref genome data

The '.faa' file is "All predicted CDS"

'fna' file is the "DNA sequence FASTA file of the genome assembly of the species representative"

A better way to download might be using the API, then we don't need to deal with the ftp address directly:
"https://www.ebi.ac.uk/metagenomics/api/v1/genomes/MGYG000000001/downloads/MGYG000000001.faa"

This is the same as using this website by hand:
https://www.ebi.ac.uk/metagenomics/genomes/MGYG000000001#downloads

However, these addresses are v2.0, so they have slightly different genome IDs: "MGYG-HGUT-00001" vs "MGYG000000001", for this reason I think we should download everything from the ftp address.

In [12]:
mgnify_accession = 'MGYG-HGUT-00001'

### Download using API

In [13]:
def accession_to_fileid(accession):
    # reformatting the id
    # my understanding is that we need to replace the HGUT part by 0000 and remove the -
    items = accession.split('-')
    return '0000'.join([items[0], items[-1]])

fileid = accession_to_fileid(mgnify_accession)
api_address = "https://www.ebi.ac.uk/metagenomics/api/v1/genomes/{}/downloads/".format(fileid)
filename = "{}.faa".format(fileid)

In [14]:
api_address + filename

'https://www.ebi.ac.uk/metagenomics/api/v1/genomes/MGYG000000001/downloads/MGYG000000001.faa'

In [15]:
import requests

# download the sequence and the annotation
filename = "{}.faa".format(fileid)
r = requests.get(api_address + filename)
open(filename, 'wb').write(r.content)

filename = "{}.fna".format(fileid)
r = requests.get(api_address + filename)
open(filename, 'wb').write(r.content)

### Download using ftp

In [16]:
# the ftp address has an upper level folder that omits the last two number of the accession id
ftp_base = "http://ftp.ebi.ac.uk/pub/databases/metagenomics/mgnify_genomes/human-gut/v1.0/uhgg_catalogue/{}/{}/genome/"
ftp_address = ftp_base.format(mgnify_accession[:-2], mgnify_accession)

In [17]:
print(ftp_address)

http://ftp.ebi.ac.uk/pub/databases/metagenomics/mgnify_genomes/human-gut/v1.0/uhgg_catalogue/MGYG-HGUT-000/MGYG-HGUT-00001/genome/


In [18]:
filename = "{}.fna".format(mgnify_accession)
r = requests.get(ftp_address + filename)
open(filename, 'wb').write(r.content)

## Now let's check the SNV table
Let's verify that the reference genome makes sense

In [19]:
# download the SNV table
ftp_address = "http://ftp.ebi.ac.uk/pub/databases/metagenomics/mgnify_genomes/human-gut/v1.0/snv_catalogue/"
mgnify_accession = 'MGYG-HGUT-00001'

filename = "{}.tsv.tar.lz4".format(mgnify_accession)
r = requests.get(ftp_address + filename)
open(filename, 'wb').write(r.content)

Decompress in bash using: 
```
lz4 -dc < MGYG-HGUT-00001.tsv.tar.lz4 | tar xvf -
```

In [20]:
import pandas as pd
snv_df = pd.read_csv('MGYG-HGUT-00001_snvs.tsv', delimiter='\t')  # using the file uploaded to github repo

In [21]:
snv_df.head(10)

,Contig,Pos,Ref,Alt,GUT_GENOME091053,GUT_GENOME178957,GUT_GENOME212267
0,GUT_GENOME000001_106,1002,C,T,1,1,1
1,GUT_GENOME000001_106,1020,T,C,1,1,1
2,GUT_GENOME000001_106,1023,C,T,1,1,1
3,GUT_GENOME000001_106,1211,A,G,1,1,1
4,GUT_GENOME000001_106,1213,T,G,1,1,1
5,GUT_GENOME000001_106,1217,T,G,1,1,1
6,GUT_GENOME000001_106,1236,A,G,1,1,1
7,GUT_GENOME000001_106,1263,C,T,1,1,1
8,GUT_GENOME000001_106,1288,T,C,1,1,1
9,GUT_GENOME000001_106,1302,C,A,1,1,1


In [22]:
# let's check the first contig
desired_contig = snv_df.iloc[0, 0]

In [23]:
from Bio import SeqIO

# parsing the fasta file for ref seqeunce
input_file = "{}.fna".format(mgnify_accession)
fasta_sequences = SeqIO.parse(open(input_file),'fasta')

all_records = []
for record in fasta_sequences:
    if record.id==desired_contig:
        desired_record = record
    all_records.append(record)
print(len(all_records))

129


In [24]:
print(desired_record.seq)
print("Contig length: {}".format(len(desired_record.seq)))

CCTAAAAATTTTATAATTTCGTTAGGCTTGTTTGCCATGCAAATTTTTAAAAATATGAAACTTAATTTAACCTATCGGTTTATTTATCAATTCGAGAATAATTCTCATCATATCTAATTAACAAAACCTAACTAAATATTTATTAATTCTAATCTTAAATCATTTAATATTGTCTTTGTTGCTCTTAATCTTATTTGCTGTCTATTTCCATTAAATACGTATCTCTTTACTATTATCTTTCCATTTATACAAATCCCAAAATATACCAAACCAACTGGTTTTTCTTCACTTCCACCTTCTGGTCCTGCTATTCCTGTTGTTGATAGCCCTATATTGGTATTATGCCTTTTAGCTATACCTTCTGCCATTTCTCTAGCTGTTTCCTCACTAACTGCTCCAAACTTATCTAGAGTTTCTTTTTTCACACCTAAAGATTTCATCTTAGCTTCATTGGAATAAGTTACACAACCTTCCATAAAAACAGATGAAATTCCTGGATAATTTATAAGGTGAGATGATACCATTCCTCCTGTACATGATTCTGCTACACTTATAGTTAAATTTTTATTAACTAAAATCTCAGCAACAGTATCTTCTAATGAAATATCACCCTCTGAATATATGTAACTTCCTACTCTATCTTGAATTTTATGTACAACTGGATTTATTAAATTAATAGCTTCTTCTTTACTAGTAGCTTTTGCTGTAATTCTTAAAGTTACTTCCGTGTCCTTTGCATATGGAGCTACAGTTGGATTTGATTGTTCTTTTATTATATCTATTATCTCTTCTTCTAAAGAAGATTCACCAATTCCATAGAATCTAAGAGTTTTAGAAACCAAAATTTCATTTGTTAAATTTTTTAAATATGGCTTAACATTTTCTTCGAACATTGGCTTCATTTCTCTTGGTGGGCCTGGTAATACTATTATTGTTTTTTTTCCTTTTCTTAAAATTGCTCCTGGTGCTGTACCATTGTTATTTTCAAGTATTATTGCAT

In [25]:
print("Downloaded ref, SNV table ref")
for _, row in snv_df.head(20).iterrows():
    # remember to change contig position to str index by subtracting 1 !!!
    # took me a while to realize this !!!
    print(desired_record.seq[int(row['Pos'])-1], row['Ref'])

Downloaded ref, SNV table ref
('C', 'C')
('T', 'T')
('C', 'C')
('A', 'A')
('T', 'T')
('T', 'T')
('A', 'A')
('C', 'C')
('T', 'T')
('C', 'C')
('A', 'A')
('C', 'C')
('C', 'C')
('A', 'A')
('T', 'T')
('C', 'C')
('A', 'A')
('T', 'T')
('A', 'A')
('A', 'A')


Now that we have checked that the reference sequence is indeed what's reported in the SNV table, the next step should be parsing the gene annotations.

## Processing the SNV tables
First step: split the SNV table by gene

In [1]:
import sys
sys.path.append('..')
import UHGG_utils

In [2]:
snvs_path = '/Volumes/Botein/uhgg/MGYG-HGUT-00001_snvs.tsv'
grouped_snvs_base = '/Volumes/Botein/uhgg/MGYG-HGUT-00001/'  # make sure to start with an empty folder
gene_df = UHGG_utils.gff_to_df('/Volumes/Botein/uhgg/MGYG-HGUT-00001.gff')

UHGG_utils.process_SNVs_table(snvs_path, gene_df, grouped_snvs_base)

Finished 0 at 19:00:13


Second step: process each individual genes and compute all pairwise counts
I've done the above for the bigger table of E. rectale

In [24]:
mgnify_accession = 'MGYG-HGUT-02492'

In [33]:
snvs_path = '/Volumes/Botein/uhgg/MGYG-HGUT-02492_snvs.tsv'
grouped_snvs_base = '/Volumes/Botein/uhgg/MGYG-HGUT-02492/'  # make sure to start with an empty folder
gene_df = UHGG_utils.gff_to_df('/Volumes/Botein/uhgg/MGYG-HGUT-02492.gff')
gene_df = gene_df.set_index('Gene ID')
genomes_metadata = pd.read_csv('/Volumes/Botein/uhgg/genomes-nr_metadata.tsv', delimiter='\t')

In [26]:
header = UHGG_utils.get_SNVs_table_header(snvs_path)

In [68]:
reload(UHGG_utils)

<module 'UHGG_utils' from '../UHGG_utils.py'>

In [79]:
from datetime import datetime
gene_files = os.listdir(grouped_snvs_base)

output_path = '/Volumes/Botein/uhgg/E_rectale_site_pairs.txt'
with open(output_path, 'w') as f:
    f.write(' '.join(('n11', 'n10', 'n01', 'n00', 'ell')) + '\n')

processed = 0

for gene_file in gene_files:
    gene_file_path = grouped_snvs_base + gene_file
    items = gene_file.split('.')[0].split('-')
    gene_id = int(items[-1])
    contig = items[0]
    if processed % 100 == 0:
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        print("Finished {} at {}".format(processed, current_time))
    if gene_df.loc[gene_id, 'Type'] != 'CDS':
        continue
    UHGG_utils.process_SNV_table_single_gene(mgnify_accession, header, genomes_metadata, gene_file_path, contig, gene_id, output_path)
    processed += 1

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Finished {} at {}".format(processed, current_time))

Finished 0 at 20:56:08
Finished 100 at 21:07:28
Finished 200 at 21:13:05
Finished 300 at 21:18:50
Finished 400 at 21:27:41
Finished 500 at 21:37:07
Finished 600 at 21:45:42
Finished 700 at 21:52:53
Finished 800 at 22:01:50
Finished 900 at 22:09:52
Finished 1000 at 22:15:21
Finished 1100 at 22:19:42
Finished 1200 at 22:27:56
Finished 1300 at 22:37:18
Finished 1400 at 22:44:55
Finished 1500 at 22:53:02
Finished 1600 at 22:58:25
Finished 1700 at 23:03:11
Finished 1800 at 23:09:45
Finished 1900 at 23:17:35
Finished 1900 at 23:17:35
Finished 2000 at 23:23:23
Finished 2100 at 23:29:20
Finished 2200 at 23:34:38
Finished 2300 at 23:38:57
Finished 2400 at 23:43:09
Finished 2500 at 23:49:06
Finished 2600 at 23:54:12
Finished 2700 at 00:01:51
Finished 2800 at 00:07:59
Finished 2800 at 00:07:59
Finished 2900 at 00:14:46
Finished 3000 at 00:19:10
Finished 3100 at 00:24:53
Finished 3200 at 00:31:40
Finished 3241 at 00:35:24


In [3]:
output_path = '/Volumes/Botein/uhgg/E_rectale_site_pairs.txt'
res = pd.read_csv(output_path, delimiter=' ')

In [11]:
res = res.to_numpy()

In [15]:
np.save('/Volumes/Botein/uhgg/E_rectale_site_pairs', res)

In [4]:
%%time
res = np.load('/Volumes/Botein/uhgg/E_rectale_site_pairs.npy')

CPU times: user 2.32 ms, sys: 6.52 s, total: 6.52 s
Wall time: 10.8 s


In [5]:
n11s = res[:, 0].astype(float)

In [6]:
n11s[0]

0.0